In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [150]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 5
DOLLAR_THRESHOLD = "115009542m"
EVENT_NAME = 'Z-SCORE-W100-1Z'
LABEL_NAME = "TB"

START_DATE = "20240101"
END_DATE = "20241231"

RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "interm/resampled"
PROCESSED_DIR = BASE_DIR / "processed"
SIDES_DIR = BASE_DIR / "interm/sides"
META_LABELS_DIR = BASE_DIR / "processed/meta_labels"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"

In [151]:
SIDE_NAME = "EMACross_9_36"
SIDE_FILE_PATH = SIDES_DIR / f"{RESAMPLED_NAME}-{SIDE_NAME}.pkl"

In [152]:
%%time
df = pd.read_pickle(RESAMPLED_FILE_PATH)
sides = pd.read_pickle(SIDE_FILE_PATH)

CPU times: user 1.15 ms, sys: 2.13 ms, total: 3.28 ms
Wall time: 2.29 ms


In [153]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.set_index('timestamp')
df.head()

,open,high,low,close,volume,spread
timestamp,,,,,,
2024-01-01 22:00:00,1.104540,1.104555,1.104505,1.104525,1.414000e+11,0.000510
2024-01-01 22:05:00,1.104515,1.104595,1.104515,1.104565,1.971000e+11,0.000595
2024-01-01 22:10:00,1.104555,1.104595,1.104545,1.104585,1.624500e+11,0.000573
2024-01-01 22:15:00,1.104580,1.104595,1.104545,1.104545,3.600000e+10,0.000577
2024-01-01 22:20:00,1.104585,1.104585,1.104440,1.104505,1.530000e+11,0.000441


In [154]:
t_events = sides.index
df.loc[t_events].head()

,open,high,low,close,volume,spread
timestamp,,,,,,
2024-01-15 06:35:00,1.096440,1.096615,1.096425,1.096595,9.697500e+11,0.000027
2024-01-15 12:05:00,1.094830,1.095140,1.094830,1.095130,1.951750e+12,0.000029
2024-01-15 14:05:00,1.094640,1.094670,1.093940,1.094110,2.123980e+12,0.000022
2024-01-15 15:55:00,1.094930,1.095095,1.094800,1.094975,2.287020e+12,0.000022
2024-01-15 20:30:00,1.095035,1.095040,1.094935,1.094990,2.535200e+11,0.000029


## Calculate Target Volatility (trgt)
This will be used for the calculation of dynamic TP/SL

In [155]:
import pandas_ta as ta

In [156]:
df.ta.atr(length=9, append=True)
df['ATRr_9_PERC'] = df.ta.atr(length=0, percent=True)
df.dropna(inplace=True)
df.head()

,open,high,low,close,volume,spread,ATRr_9,ATRr_9_PERC
timestamp,,,,,,,,
2024-01-01 23:10:00,1.104040,1.104045,1.103845,1.103845,3.608200e+11,0.000036,0.000122,0.009900
2024-01-01 23:15:00,1.103840,1.103845,1.103735,1.103735,2.821800e+11,0.000037,0.000120,0.009906
2024-01-01 23:20:00,1.103745,1.103820,1.103725,1.103775,5.239900e+11,0.000043,0.000118,0.009813
2024-01-01 23:25:00,1.103770,1.103835,1.103735,1.103775,3.200500e+11,0.000035,0.000116,0.009759
2024-01-01 23:30:00,1.103780,1.103790,1.103630,1.103675,2.934700e+11,0.000028,0.000121,0.010098


In [157]:
trgt = df['ATRr_9_PERC'].reindex(t_events)

In [158]:
trgt.describe()

count    2558.000000
mean        0.025420
std         0.014600
min         0.004569
25%         0.014902
50%         0.022162
75%         0.031466
max         0.124161
Name: ATRr_9_PERC, dtype: float64

## Apply Triple Barrier
We use daily volatility to calculate TP/SL, and we set our T1 as 1day

In [159]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd

def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    pt = ptSl[0] * events_['trgt'] if ptSl[0] > 0 else pd.Series(index=events.index)
    sl = -ptSl[1] * events_['trgt'] if ptSl[1] > 0 else pd.Series(index=events.index)

        
    for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
        df0=close[loc:t1] # path prices, i.e. price[t0:t1]
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
    return out


def parallel_apply(func, items, num_threads=4, **kwargs):
    
    def worker(molecule):
        return func(molecule=molecule, **kwargs)

    chunks = np.array_split(items, num_threads)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return pd.concat(results).sort_index()


def getEvents(close, tEvents, ptSl, trgt, minTrgt, numThreads=4, t1=False, side=None):
    # Step 1: Filter targets
    trgt = trgt.loc[tEvents]
    trgt = trgt[trgt > minTrgt]

    # Step 2: Set vertical barrier (t1)
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)

    # Step 3: Build events DataFrame
    if side is None:
        side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
    else:
        side_, ptSl_ = side.loc[trgt.index], ptSl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])

    # Step 4: Apply barriers in parallel
    df0 = parallel_apply(
        func=applyPtSlOnT1,
        items=events.index,
        num_threads=numThreads,
        close=close,
        events=events,
        ptSl=ptSl_
    )

    # Step 5: Choose the first touched barrier
    events['t1'] = df0.dropna(how='all').min(axis=1)
    
    if side is None:
        events = events.drop('side', axis=1)
        
    return events


In [160]:
def get_vertical_barrier(t_events, close, delta=pd.Timedelta(minutes=60)):
    """
    For each event in t_events, find the timestamp in close.index 
    that is at least delta later. Returns a pd.Series of t1.
    """
    barrier_times = t_events + delta
    t1_idx = close.index.searchsorted(barrier_times)
    valid_idx = t1_idx[t1_idx < len(close)]
    t1 = pd.Series(close.index[valid_idx], index=t_events[:len(valid_idx)])
    return t1

In [161]:
t1 = get_vertical_barrier(t_events, df['close'], delta=pd.Timedelta(minutes=180))

In [162]:
t1

timestamp
2024-01-15 06:35:00   2024-01-15 09:35:00
2024-01-15 12:05:00   2024-01-15 15:05:00
2024-01-15 14:05:00   2024-01-15 17:05:00
2024-01-15 15:55:00   2024-01-15 18:55:00
2024-01-15 20:30:00   2024-01-15 23:30:00
                              ...        
2024-12-30 06:00:00   2024-12-30 09:00:00
2024-12-30 07:25:00   2024-12-30 10:25:00
2024-12-30 09:50:00   2024-12-30 12:50:00
2024-12-30 13:25:00   2024-12-30 16:25:00
2024-12-30 18:05:00   2024-12-30 21:05:00
Name: timestamp, Length: 2558, dtype: datetime64[ns]

In [163]:
trgt.describe()

count    2558.000000
mean        0.025420
std         0.014600
min         0.004569
25%         0.014902
50%         0.022162
75%         0.031466
max         0.124161
Name: ATRr_9_PERC, dtype: float64

In [164]:
minTrgt = trgt.quantile(.3)
minTrgt = 0

In [165]:
%%time

events = getEvents(
    close=df['close'],
    tEvents=t_events, 
    ptSl=[.5,.5], 
    trgt=trgt, 
    minTrgt=minTrgt, 
    numThreads=20, 
    t1=t1, 
    side=sides
)

CPU times: user 571 ms, sys: 7.67 ms, total: 579 ms
Wall time: 574 ms


In [166]:
events.head()

,t1,trgt,side
timestamp,,,
2024-01-15 06:35:00,2024-01-15 09:35:00,0.014592,0
2024-01-15 12:05:00,2024-01-15 15:05:00,0.031122,1
2024-01-15 14:05:00,2024-01-15 17:05:00,0.038892,0
2024-01-15 15:55:00,2024-01-15 18:55:00,0.032986,1
2024-01-15 20:30:00,2024-01-15 23:30:00,0.009128,0


In [167]:
events = events.dropna().copy()

# Keep only intraday events

In [168]:
events = events[events.index.date==events.t1.dt.date]

## Create classification labels

In [169]:
def getBins(events, close, minRet=0.0, t1=None):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    -t1 is original vertical barrier series
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    # 1) prices aligned with events
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # 2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[
        events_.index] - 1
    if 'side' in events_: out['ret'] *= events_['side']  # meta-labeling
    out['bin'] = np.sign(out['ret'])

    if 'side' not in events_:
        # only applies when not meta-labeling.
        # to update bin to 0 when vertical barrier is touched, we need the
        # original vertical barrier series since the events['t1'] is the time
        # of first touch of any barrier and not the vertical barrier
        # specifically. The index of the intersection of the vertical barrier
        # values and the events['t1'] values indicate which bin labels needs
        # to be turned to 0.
        vtouch_first_idx = events[events['t1'].isin(t1.values)].index
        out.loc[vtouch_first_idx, 'bin'] = 0.

    if 'side' in events_: out.loc[out['ret'] <= 0, 'bin'] = 0  # meta-labeling
    # out.loc[out['ret'].abs() < minRet, 'bin'] = 0. # 不做獲利過少的交易 (設為 0)
    # out = out.loc[out['ret'].abs() >= minRet] # 過濾掉獲利過少的交易 (刪掉)
    return out

In [170]:
labels = getBins(events, df['close'], minRet=0, t1=t1)

In [171]:
labels.bin.value_counts()

bin
0.0    1626
1.0     567
Name: count, dtype: int64

In [185]:
labels = labels.join(events)

In [186]:
LABELS_FILE_PATH = META_LABELS_DIR / f"{RESAMPLED_NAME}-{SIDE_NAME}-meta.pkl"

In [188]:
labels.to_pickle(LABELS_FILE_PATH)